In [1]:
from fastai import *
from fastai.text import *


Bad key "text.kerning_factor" on line 4 in
/home/pranav/anaconda3/envs/env-TF2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
%matplotlib inline

In [3]:
import sklearn.feature_extraction.text as sklearn_text

In [4]:
??URLs

In [5]:
URLs.IMDB_SAMPLE

'http://files.fast.ai/data/examples/imdb_sample'

In [6]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/pranav/.fastai/data/imdb_sample')

In [7]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [8]:
movie_reviews = (TextList.from_csv(path,'texts.csv', cols= 'text').split_from_df(col=2).label_from_df(cols=0))

In [9]:
movie_reviews.valid

LabelList (200 items)
x: TextList
xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
 
  xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
 
  xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , x

In [10]:
??TextList

In [11]:
len(movie_reviews.train.x)

800

In [12]:
#Counter _> creates dictionary of frquencies
c = Counter([1,1,1,2,3,4,4,5,3])
c

Counter({1: 3, 2: 1, 3: 2, 4: 2, 5: 1})

In [13]:
# storing values of amtrix
# Coodinat wise_> row , col , val
# Compressed Sparse row method 

In [14]:
# construct the document-term matrix in CSR format
# i.e. return (values, column_indices, row_pointer)
def get_ter_doc_mat(text_list, n_terms):
    
    # inputs:
    #    text_list, a TextList object
    #    n_terms, the number of tokens in our IMDb vocabulary
    
    # output: 
    #    the CSR format sparse representation of the document-term matrix in the form of a
    #    scipy.sparse.csr.csr_matrix object

    
    # initialize arrays
    values = []
    column_indices = []
    row_pointer = []
    row_pointer.append(0)

    # from the TextList object
    for _, doc in enumerate(text_list):
        feature_counter = Counter(doc.data)
        column_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        # Tack on N (number of nonzero elements in the matrix) to the end of the row_pointer array
        row_pointer.append(len(values))
        
    return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                   shape=(len(row_pointer) - 1, n_terms),
                                   dtype=int)

In [15]:
%%time
val_term_doc = get_ter_doc_mat(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

CPU times: user 58.2 ms, sys: 4.3 ms, total: 62.5 ms
Wall time: 58.5 ms


In [16]:
val_term_doc[:10, :10]

<10x10 sparse matrix of type '<class 'numpy.int64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [17]:
train_term_doc = get_ter_doc_mat(movie_reviews.train.x , len(movie_reviews.vocab.itos))

In [18]:
train_term_doc.shape

(800, 6008)

In [19]:
val_term_doc.shape

(200, 6008)

In [20]:
val_term_doc

<200x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 27848 stored elements in Compressed Sparse Row format>

In [21]:
val_term_doc[1]

<1x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 81 stored elements in Compressed Sparse Row format>

In [22]:
dir(movie_reviews)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_test',
 'add_test_folder',
 'databunch',
 'filter_by_func',
 'get_processors',
 'label_const',
 'label_empty',
 'label_from_df',
 'label_from_folder',
 'label_from_func',
 'label_from_list',
 'label_from_lists',
 'label_from_re',
 'lists',
 'load_empty',
 'load_state',
 'path',
 'process',
 'test',
 'train',
 'transform',
 'transform_y',
 'valid']

In [23]:
movie_reviews.y.classes

['negative', 'positive']

In [24]:
X = train_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [25]:
dir(movie_reviews.train)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'c',
 'databunch',
 'export',
 'filter_by_func',
 'get_state',
 'item',
 'load_empty',
 'load_state',
 'new',
 'predict',
 'process',
 'set_item',
 'tfm_y',
 'tfmargs',
 'tfmargs_y',
 'tfms',
 'tfms_y',
 'to_csv',
 'to_df',
 'transform',
 'transform_y',
 'x',
 'y']

In [26]:
movie_reviews.c

2

In [27]:
movie_reviews.train.y.c2i

{'negative': 0, 'positive': 1}

In [28]:
positive = y.c2i['positive']
negative = y.c2i['negative']


In [29]:
v = movie_reviews.vocab

In [30]:
p1 = np.squeeze(np.asarray(X[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(X[y.items==negative].sum(0)))

In [31]:
v.stoi['love']

142

In [32]:
p1[142]

133

In [33]:
p0[142]

75

In [34]:
#Applying naive bayes

In [35]:
P1 = (p1 + 1)/((y.items==positive).sum()+ 1)
P0 = (p0 + 1)/((y.items==negative).sum()+ 1)

#### From the class likelihood ratios, we can define a **log-count ratio** $R_{t}$ for each token $t$ as
### $ R_{t} = \text{log} \frac{L(t|+)}  {L(t|-)}$
#### The `log-count ratio` ranks tokens by their relative affinities for positive and negative reviews
#### We observe that
* $R_{t} \gt 0$ means `positive` reviews are more likely to contain this token 
* $R_{t} \lt 0$ means `negative` reviews are more likely to contain this token 
* $R_{t} = 0$ indicates the token $t$ has equal likelihood to appear in  `positive` and `negative` reviews


In [36]:
r = np.log(P1/P0); r

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

In [37]:
n_tokens = 10
highest_R = np.argpartition(r, -n_tokens)[-n_tokens:] # positive
lowest_R = np.argpartition(r, n_tokens)[:n_tokens]    # negative

In [38]:
highest_R

array([1723, 1662, 1666, 1212, 1143, 1529, 1386, 1620, 1358,  795])

In [39]:
b = np.log((y.items==positive).mean()  / (y.items==negative).mean()); b

-0.08505123261815539

In [40]:
pred = (val_term_doc @ r + b) > 0

In [41]:
(pred ==val_y.items).mean()  ## accuracy

0.645

In [42]:
# Now using full dataset

In [43]:
path = untar_data(URLs.IMDB)

In [44]:
path.ls()

[PosixPath('/home/pranav/.fastai/data/imdb/train'),
 PosixPath('/home/pranav/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/pranav/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/pranav/.fastai/data/imdb/tmp_lm'),
 PosixPath('/home/pranav/.fastai/data/imdb/README'),
 PosixPath('/home/pranav/.fastai/data/imdb/test'),
 PosixPath('/home/pranav/.fastai/data/imdb/unsup')]

In [45]:
(path/'train').ls()

[PosixPath('/home/pranav/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/home/pranav/.fastai/data/imdb/train/labeledBow.feat'),
 PosixPath('/home/pranav/.fastai/data/imdb/train/neg'),
 PosixPath('/home/pranav/.fastai/data/imdb/train/pos')]

In [46]:
%%time
# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        reviews_full = (TextList.from_folder(path)
             #  Make a `TextList` object that is a list of `WindowsPath` objects, 
             #     each of which contains the full path to one of the data files.
             .split_by_folder(valid='test')
             # Generate a `LabelLists` object that splits files by training and validation folders
             # Note: .label_from_folder in next line causes the `BrokenProcessPool` error
             .label_from_folder(classes=['neg', 'pos']))
             # Create a `CategoryLists` object which contains the data and
             #   its labels that are derived from folder names
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')


failure count is 0

CPU times: user 25 s, sys: 10.6 s, total: 35.6 s
Wall time: 10min 31s


In [47]:
v = reviews_full.vocab

In [48]:
%%time
valid_doc_term = get_ter_doc_mat(reviews_full.valid.x, len(reviews_full.vocab.itos))

CPU times: user 5.38 s, sys: 128 ms, total: 5.51 s
Wall time: 6.68 s


In [49]:
train_doc_term = get_ter_doc_mat(reviews_full.train.x, len(reviews_full.vocab.itos))

In [50]:
# saving these doc_term matrices
scipy.sparse.save_npz('valid_doc_term.npz' , valid_doc_term)
scipy.sparse.save_npz('train_doc_term.npz' , train_doc_term)

In [51]:
# Naive bayes on full dataset

In [52]:
x = train_doc_term
y = reviews_full.train.y

val_y = reviews_full.valid.y.items

In [53]:
x

<25000x38456 sparse matrix of type '<class 'numpy.int64'>'
	with 3716265 stored elements in Compressed Sparse Row format>

In [54]:
positive = y.c2i['pos']
negative = y.c2i['neg']

In [55]:
C0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
C1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [56]:
L1 = (C1+1) / ((y.items==positive).sum() + 1)
L0 = (C0+1) / ((y.items==negative).sum() + 1)

In [57]:
R = np.log(L1/L0); R

array([0.066783, 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ])

In [58]:
b = np.log((y.items==positive).mean()  / (y.items==negative).mean()); b

0.0

In [59]:
predictions = (valid_doc_term @ R + b) > 0 ; predictions

array([False, False, False, False, ..., False,  True, False, False])

In [60]:
(predictions == val_y).mean() #80 % accuracy

0.8084

In [76]:
#Binarized Naive bayes

x= train_doc_term.sign()
x

<25000x38456 sparse matrix of type '<class 'numpy.int64'>'
	with 3716265 stored elements in Compressed Sparse Row format>

In [62]:
C0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
C1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [63]:
L1 = (C1+1) / ((y.items==positive).sum() + 1)
L0 = (C0+1) / ((y.items==negative).sum() + 1)

In [64]:
R = np.log(L1/L0)
b = np.log((y.items==positive).mean()  / (y.items==negative).mean())

In [66]:
predictions = (valid_doc_term.sign() @ R + b) > 0 ; predictions

array([False, False, False, False, ..., False,  True, False, False])

In [68]:
(predictions == val_y).mean() #82.9 % accuracy

0.82924

In [69]:
# Logistic Regression

In [70]:
from sklearn.linear_model import LogisticRegression

In [100]:
m = LogisticRegression(C= 0.1,dual = False, solver = 'liblinear')
m.fit(x.sign() ,y.items.astype(int))
preds = m.predict(valid_doc_term)
(preds == val_y).mean()

0.7704